# Notebook about integrating GECKO models into DD-DeCaF

This notebook contains three stages of the integration of enzyme constrain models (ecModels) to the platform:
1. Test of ecModel of *Escherichia coli* eciML1515.
2. Functions to be added to `driven.py`.
3. Test of these functions.

## 1. Testing the model of E. coli

As a controlled-version model, it can be downloaded from the [GitHub](https://github.com/SysBioChalmers/ecModels/blob/chore/updateiML1515/eciML1515/model/eciML1515.xml) repository. We're using a modified version with bounds in a cobrapy-like manner.

In [1]:
! [ ! -f "eciML1515.xml" ] && curl -O -L "https://raw.githubusercontent.com/BenjaSanchez/notebooks/master/eciML1515.xml"

In [2]:
import cobra

model = cobra.io.read_sbml_model("eciML1515.xml")

Notice here that the actual model to be uploaded in the DD-DeCaF platform will be in `JSON` format.
Now, it is time to test the model and see if its behavior is fine.

In [3]:
model.objective.expression.args[0]

1.0*BIOMASS_Ec_iML1515_core_75p37M

In [4]:
model.solver.timeout = 15
model.optimize()

,fluxes,reduced_costs
EX_acgam_e,0.000000,-0.282126
EX_cellb_e,0.000000,-0.361121
EX_chol_e,0.000000,-0.022570
EX_pi_e,0.000000,0.000000
EX_h_e,8.058201,0.000000
...,...,...
prot_Q59385_exchange,0.000000,-0.000000
prot_Q6BEX0_exchange,0.000000,-0.000000
prot_Q6BF16_exchange,0.000000,-0.000000
prot_Q6BF17_exchange,0.000000,-0.000000


In [5]:
# Aerobic growth
with model as mod:
    mod.reactions.EX_o2_e.lower_bound = -10
    solution_anaerobic_ec = mod.optimize()
solution_anaerobic_ec

,fluxes,reduced_costs
EX_acgam_e,0.000000,-0.282126
EX_cellb_e,0.000000,-0.361121
EX_chol_e,0.000000,-0.022570
EX_pi_e,0.000000,0.000000
EX_h_e,8.058201,0.000000
...,...,...
prot_Q59385_exchange,0.000000,-0.000000
prot_Q6BEX0_exchange,0.000000,-0.000000
prot_Q6BF16_exchange,0.000000,-0.000000
prot_Q6BF17_exchange,0.000000,-0.000000


Since I haven't been capable of making the EC model to work, I'll try now with its yeast counterpart.

## 1.1 Testing the ecYeast model in cobrapy
Accordingly, the yeast model can be obtained from its [GitHhub repository](https://github.com/SysBioChalmers/ecModels/tree/master/ecYeastGEM/model).

In [6]:
! [ ! -f "ecYeastGEM.xml" ] && curl -O -L "https://raw.githubusercontent.com/SysBioChalmers/ecModels/master/ecYeastGEM/model/ecYeastGEM.xml"

In [7]:
yeast = cobra.io.read_sbml_model("ecYeastGEM.xml")

Let's make the same tests.

In [8]:
yeast.reactions.r_2111

Reaction identifier,r_2111
Name,growth
Memory address,0x07efe5cee8940
Stoichiometry,s_0450__91__c__93__ --> biomass [cytoplasm] -->
GPR,
Lower bound,0.0
Upper bound,inf


In [9]:
yeast.objective.expression

1.0*r_2111 - 1.0*r_2111_reverse_58b69

In [10]:
yeast.solver.timeout = 10
yeast.optimize()

,fluxes,reduced_costs
r_0006,0.022002,0.000000
r_0070,0.000000,0.000000
r_0094,0.000000,0.000000
r_0099,0.000000,0.000000
r_0200,0.000000,-0.041053
...,...,...
prot_Q99190_exchange,0.000000,-0.000000
prot_Q99258_exchange,0.000000,-0.000000
prot_Q99288_exchange,0.000000,-0.000000
prot_Q99312_exchange,0.000000,0.000000


In [11]:
id_o2 = yeast.exchanges.query('oxygen exchange', 'name')[0].id
# anaerobic growth
yeast.reactions.get_by_id('r_1992').bounds = 0,0
yeast.optimize()

,fluxes,reduced_costs
r_0006,0.022002,0.000000
r_0070,0.000000,0.000000
r_0094,0.000000,0.000000
r_0099,0.000000,0.000000
r_0200,0.000000,-0.041053
...,...,...
prot_Q99190_exchange,0.000000,-0.000000
prot_Q99258_exchange,0.000000,-0.000000
prot_Q99288_exchange,0.000000,-0.000000
prot_Q99312_exchange,0.000000,0.000000


In [12]:
import math

# this was proven to be unnecessary 
for reac in yeast.reactions:
    if math.isinf(reac.upper_bound):
        reac.upper_bound = 1000

## 2. Functions to integrate the model in `adapter.py`

This functions are called in the order they appear in the file. They are thought to be consistent with the fluxomics implementation.

In [13]:
import pandas as pd
import re
from math import isnan

prot_pat = re.compile(r'prot_([A-Za-z0-9]+)__91__c__93__')

def flexibilize_proteomics(model, biomass_reaction, growth_rate, proteomics):
    """
    Replace proteomics measurements with the set of proteomics measures that enables
    the model to achieve growth.

    Proteins are removed from the set iteratively based on sensitivity analysis.

    Parameters
    ----------
    model: cobra.Model
    biomass_reaction: str
        The id of the biomass reaction in the goperationsiven model.
    proteomics: list(dict)
        List of measurements matching the `Proteomics` schema.
    growth_rate: dict
        Growth rate, matching the `GrowthRate` schema.

    Returns
    -------
    growth_rate: dict
    proteomics: list(dict)
    """
    # TODO: this whole thing about growth rate could be refactored, since it's exactly
    # the same as in `minimize_distance`
    if not growth_rate:
        raise ValueError(
            "Expected measurements to contain an objective "
            "constraint as measured growth rate"
        )

    if growth_rate["uncertainty"]:
        lower_bound = growth_rate["measurement"] - growth_rate["uncertainty"]
        upper_bound = growth_rate["measurement"] + growth_rate["uncertainty"]
    else:
        lower_bound = growth_rate["measurement"]
        upper_bound = growth_rate["measurement"]

    model.reactions.get_by_id(biomass_reaction).bounds = (lower_bound, upper_bound)
    
    index, observations = [], []
    
    for measure in proteomics:
        index.append(measure["identifier"])
        # in protein data, upper_bound is the unique constraint, so it contains the uncertainty
        observations.append(measure["measurement"] + measure["uncertainty"])

    observations = pd.Series(index=index, data=observations)

    solution, new_growth_rate = ensure_proteomics_growing(model, observations)
    if new_growth_rate:
        growth_rate["measurement"] = new_growth_rate
    for reaction, new_measurement in solution.iteritems():
        for measure in proteomics:
            if reaction == measure.get("identifier"):
                measure["measurement"] = new_measurement
                measure["uncertainty"] = 0  # TODO: Confirm that this is correct
    return growth_rate, proteomics


def limit_proteins(model, measured_mmolgdw):
    """Apply proteomics measurements to model.

    Apply measurements in the form of measured proteins.

    Parameters
    ----------
    model: cobra.Model
    measured_mmolgdw : pd.Series
        Protein abundances in g / gDW

    References
    ----------
    .. [1] Benjamin J. Sanchez, Cheng Zhang, Avlant Nilsson, Petri-Jaan Lahtvee, Eduard J. Kerkhoven, Jens Nielsen (
       2017). Improving the phenotype predictions of a yeast genome-scale metabolic model by incorporating enzymatic
       constraints. [Molecular Systems Biology, 13(8): 935, http://www.dx.doi.org/10.15252/msb.20167411
    """
    # update upper_bound
    for protein_id, measure in measured_mmolgdw.iteritems():
        try:
            rxn = model.reactions.get_by_id("prot_{}_exchange".format(protein_id))
            rxn.annotation["uniprot"] = protein_id
        except KeyError:
            pass
        else:
            rxn.bounds = 0, measure
    # flexibilize proteomics so model grows
    return


def ensure_proteomics_growing(model, measured_mmolgdw):
    """
    Optimize the model and flexiblize proteins until it grows

    Parameters
    ----------
    model: cobra.Model
    measured_mmolgdw: pd.Series
        Protein abundances in mmol / gDW

    Returns
    -------
    measured_mmolgdw: list(dict)
    obj_value: float. False if it wasn't changed
    """
    # first, get shadow prices of the unconstrained model
    solution = model.optimize()
    ordered_proteins = list(
        top_protein_shadow_prices(
            solution, measured_mmolgdw.index, len(measured_mmolgdw)
        ).index
    )
    # second, constrain the model
    with model as mod:
        limit_proteins(mod, measured_mmolgdw)
        obj_value = mod.slim_optimize()
    tolerance = 1e-7
    obj_value = False if math.isnan(obj_value) else obj_value

    # while the model can't grow, unconstrain the protein with the higher shadow price
    while obj_value < tolerance and not measured_mmolgdw.empty:
        uniprot_id = re.sub(prot_pat, r"\1", ordered_proteins[0])
        if uniprot_id in measured_mmolgdw:
            del measured_mmolgdw[uniprot_id]
        ordered_proteins.pop(0)
        with model as mod:
            # this can be changed to simply affect the upper_bound
            # but this way is more clear
            limit_proteins(mod, measured_mmolgdw)
            obj_value = mod.slim_optimize()
        obj_value = False if math.isnan(obj_value) else obj_value

    return measured_mmolgdw, obj_value

def top_protein_shadow_prices(model_solution, set_proteins, top=1):
    """
    Retrieves `top` of proteins in `set_proteins` in terms of influence in the objective
    function (shadow prices).

    Parameters
    ----------
    model_solution: cobra.Solution
        the usual Solution object returned by model.optimize()
    set_proteins: iterable of strings
        Uniprot IDs of the proteins in measurements
    top: int
        the number of proteins to be returned
    """
    set_as_metabolites = {"prot_{}__91__c__93__".format(prot) for prot in set_proteins}
    shadow_pr = model_solution.shadow_prices
    return shadow_pr.loc[shadow_pr.index.isin(set_as_metabolites)].sort_values()[:top]

Also, as noted in the functions above, we need a new class included on the `schemas.py` file.

In [14]:
from marshmallow import Schema, fields

class Proteomics(Schema):
    identifier = fields.String(required=True)  # protein UniProt code
    measurement = fields.Float(required=True)
    uncertainty = fields.Float(required=True)
    
class GrowthRate(Schema):
    measurement = fields.Float(required=True)
    uncertainty = fields.Float(required=True)

## 3. Test of the functions

An `apply_measurements`-like function to see if it works.

In [15]:
from cobra.io.dict import reaction_to_dict


def apply_measurements(
    model,
    biomass_reaction,
    proteomics,
    growth_rate,
):
    """
    Apply omics measurements to a metabolic model.

    For each measured flux (production-rate / uptake-rate), constrain the model
    by forcing their upper and lower bounds to the measured values.

    Parameters
    ----------
    model: cobra.Model
    biomass_reaction: str
        The id of the biomass reaction in the given model.
    fluproteomicsxomics: list(dict)
        List of measurements matching the `Proteomics` schema.
    growth_rate: dict
        Growth rate, matching the `GrowthRate` schema.

    Returns
    -------
    tuple (operations, warnings, errors)
        Operations is a list of model operations necessary to apply the
        measurements to the model. See also the `Operations` schema.
        Warnings is a list of human-readable strings of potential issues.
        If errors is not an empty list, it was not possible to apply the
        measurements.
        Errors then contains a list of string messages describing the
        problem(s).
    """
    operations = []
    warnings = []
    errors = []

    def bounds(measurement, uncertainty):
        """Return resolved bounds based on measurement and uncertainty"""
        if uncertainty:
            return (measurement - uncertainty, measurement + uncertainty)
        else:
            return (measurement, measurement)
    # The proteomics are added only if we have an enzyme constraint model.
    # TODO: check if ecModel before applying proteomics. It should be notified in user-interface.
    if growth_rate:
        growth_rate, proteomics = flexibilize_proteomics(
            model, biomass_reaction, growth_rate, proteomics
        )

    # Constrain the model with the observed growth rate
    if growth_rate:
        reaction = model.reactions.get_by_id(biomass_reaction)
        reaction.bounds = bounds(growth_rate["measurement"], growth_rate["uncertainty"])
        operations.append(
            {
                "operation": "modify",
                "type": "reaction",
                "id": reaction.id,
                "data": reaction_to_dict(reaction),
            }
        )
    
    LB = 0 # for all protein_exchanges
    for measure in proteomics:
        try:
            reaction = model.reactions.get_by_id("prot_{}_exchange".format(measure["identifier"]))
        except KeyError:
            errors.append(
                f"Cannot find reaction '{measure['identifier']}' in the model"
            )
        else:
            # uncertainty was taken into account in the upper bounds in `flexibilize_proteomics`
            reaction.bounds = LB, measure["measurement"]
            operations.append(
                {
                    "operation": "modify",
                    "type": "reaction",
                    "id": reaction.id,
                    "data": reaction_to_dict(reaction)
                }
            )
    return operations, warnings, errors

### 3.1. E. coli_ model
First, prepare random data to act as measurements for measurements.

In [16]:
from random import choices

prot_exchanges = [reac for reac in model.reactions if reac.id.startswith("prot_") and not reac.id.endswith("REV")]
picked_proteins = {reac.id[5:-9]: 0.01 for reac in choices(prot_exchanges, k=3)}
picked_proteins

{'P04693': 0.01, 'P0A836': 0.01, 'P0A9P9': 0.01}

In [17]:
proteomics = []
prot_schema = Proteomics()
proteomics = [prot_schema.dump(dict(identifier = idx, measurement=val, uncertainty = 0.0001))
              for idx, val in picked_proteins.items()]
proteomics

[{'measurement': 0.01, 'uncertainty': 0.0001, 'identifier': 'P04693'},
 {'measurement': 0.01, 'uncertainty': 0.0001, 'identifier': 'P0A836'},
 {'measurement': 0.01, 'uncertainty': 0.0001, 'identifier': 'P0A9P9'}]

In [18]:
growth_rate = GrowthRate().dump(dict(measurement=0.87, uncertainty=0.0001))
growth_rate

{'uncertainty': 0.0001, 'measurement': 0.87}

In [19]:
with model as mod:
    print(apply_measurements(mod, 'BIOMASS_Ec_iML1515_core_75p37M', proteomics, growth_rate))

([{'operation': 'modify', 'type': 'reaction', 'id': 'BIOMASS_Ec_iML1515_core_75p37M', 'data': OrderedDict([('id', 'BIOMASS_Ec_iML1515_core_75p37M'), ('name', 'biomass pseudoreaction'), ('metabolites', OrderedDict([('10fthf_c__91__c__93__', -0.000223), ('2fe2s_c__91__c__93__', -2.6e-05), ('2ohph_c__91__c__93__', -0.000223), ('4fe4s_c__91__c__93__', -0.00026), ('adp_c__91__c__93__', 75.3772), ('ala__L_c__91__c__93__', -0.513689), ('amet_c__91__c__93__', -0.000223), ('arg__L_c__91__c__93__', -0.295792), ('asn__L_c__91__c__93__', -0.241055), ('asp__L_c__91__c__93__', -0.241055), ('atp_c__91__c__93__', -75.5522), ('btn_c__91__c__93__', -2e-06), ('ca2_c__91__c__93__', -0.005205), ('cl_c__91__c__93__', -0.005205), ('coa_c__91__c__93__', -0.000576), ('cobalt2_c__91__c__93__', -2.5e-05), ('ctp_c__91__c__93__', -0.133508), ('cu2_c__91__c__93__', -0.000709), ('cys__L_c__91__c__93__', -0.09158), ('datp_c__91__c__93__', -0.026166), ('dctp_c__91__c__93__', -0.027017), ('dgtp_c__91__c__93__', -0.0270

The last thing we want to check is to solve the same test but now with one essential protein set to 0.

In [20]:
model.solver.timeout = 10 # enough
# essential protein to 0
proteomics.append(prot_schema.dump(dict(identifier = 'O32583', measurement=0, uncertainty = 0.0)))
apply_measurements(mod, 'BIOMASS_Ec_iML1515_core_75p37M', proteomics, growth_rate)

([{'operation': 'modify',
   'type': 'reaction',
   'id': 'BIOMASS_Ec_iML1515_core_75p37M',
   'data': OrderedDict([('id', 'BIOMASS_Ec_iML1515_core_75p37M'),
                ('name', 'biomass pseudoreaction'),
                ('metabolites',
                 OrderedDict([('10fthf_c__91__c__93__', -0.000223),
                              ('2fe2s_c__91__c__93__', -2.6e-05),
                              ('2ohph_c__91__c__93__', -0.000223),
                              ('4fe4s_c__91__c__93__', -0.00026),
                              ('adp_c__91__c__93__', 75.3772),
                              ('ala__L_c__91__c__93__', -0.513689),
                              ('amet_c__91__c__93__', -0.000223),
                              ('arg__L_c__91__c__93__', -0.295792),
                              ('asn__L_c__91__c__93__', -0.241055),
                              ('asp__L_c__91__c__93__', -0.241055),
                              ('atp_c__91__c__93__', -75.5522),
                          

### 3.2. Yeast model

First, prepare the proteomics and growth rate measurements

In [21]:
some_measurements = {'P00549': 0.01, 'P31373': 0.01, 'P31382': 0.011}
proteomics = []
prot_schema = Proteomics()
proteomics = [prot_schema.dump(dict(identifier = idx, measurement=val, uncertainty = 0.0001))
              for idx, val in some_measurements.items()]
print(proteomics)

# now, the growth_rate
growth_rate = GrowthRate().dump(dict(measurement=0.088, uncertainty=0.0001))
print("\n",growth_rate)

[{'measurement': 0.01, 'uncertainty': 0.0001, 'identifier': 'P00549'}, {'measurement': 0.01, 'uncertainty': 0.0001, 'identifier': 'P31373'}, {'measurement': 0.011, 'uncertainty': 0.0001, 'identifier': 'P31382'}]

 {'uncertainty': 0.0001, 'measurement': 0.088}


In [22]:
with yeast as mod:
    apply_measurements(mod, 'r_2111', proteomics, growth_rate)

In [23]:
# essential protein to 0
proteomics.append(prot_schema.dump(dict(identifier = 'P00498', measurement=0, uncertainty = 0.0)))
apply_measurements(yeast, 'r_2111', proteomics, growth_rate)

([{'operation': 'modify',
   'type': 'reaction',
   'id': 'r_2111',
   'data': OrderedDict([('id', 'r_2111'),
                ('name', 'growth'),
                ('metabolites', OrderedDict([('s_0450__91__c__93__', -1.0)])),
                ('lower_bound', 0.08787409974594668),
                ('upper_bound', 0.08807409974594668),
                ('gene_reaction_rule', ''),
                ('objective_coefficient', 1.0),
                ('annotation',
                 OrderedDict([('metanetx.reaction', 'MNXR137261')]))])},
  {'operation': 'modify',
   'type': 'reaction',
   'id': 'prot_P00549_exchange',
   'data': OrderedDict([('id', 'prot_P00549_exchange'),
                ('name', 'prot_P00549_exchange'),
                ('metabolites',
                 OrderedDict([('prot_P00549__91__c__93__', 1.0)])),
                ('lower_bound', 0),
                ('upper_bound', 0.0101),
                ('gene_reaction_rule', 'YAL038W'),
                ('annotation', OrderedDict([('uniprot',

## Cleaning

In [24]:
import os

os.remove("eciML1515.xml")
os.remove("ecYeastGEM.xml")